In [1]:
import pandas as pd
import numpy as np

In [2]:
polyfun = pd.read_csv('polyfun_eval_results.csv')

In [3]:
polyfun

,response,predictors,metric,eval,l_eval,u_eval,P,n,pheno_id
0,PHENO,PRS,R2,0.001469,-0.003085,0.006023,2.086498e-01,1078,INI1003063
1,PHENO,PRS,R2,0.024756,0.006918,0.042594,1.058424e-07,1130,INI20030780
2,PHENO,PRS,R2,0.001726,-0.003045,0.006496,1.584734e-01,1154,INI30120
3,PHENO,PRS,R2,0.005557,-0.003046,0.014160,1.203659e-02,1134,INI50030700


In [5]:
PRISM_dir = '/home/lucytian/data/1_Single_Cell_PRS/2_cV2F'
data_d = 'mvp_afr_0.9_0.01/20231221/406k_geno_v2_UKB_18PCs/fit_w_val/'


tissue = ['BLOOD', 'LIVER', 'LUNG', 'KIDNEY', 'all']
trait = pd.read_csv(f'{PRISM_dir}/pheno_tissue.txt', header=None)[0].tolist()

df_evals = []
for ti in tissue:
    for tr in trait:
        #print(data_d + ti + '/' + tr + '/')
        if tr =='INI20030780':
            df = pd.read_csv(PRISM_dir + '/' + data_d + ti + '/' + tr + '/exclude_APOE/snpnet.eval.tsv.gz', sep='\t', compression='gzip')
        else:
            df = pd.read_csv(PRISM_dir + '/' + data_d + ti + '/' + tr + '/snpnet.eval.tsv.gz', sep='\t', compression='gzip',)
        df['Tissue'] = [ti] * len(df)
        df.replace(np.nan, 'All', inplace=True)
        df_evals.append(df)

In [7]:
df_full = pd.concat(df_evals)
filtered = df_full[(df_full['population'] == 'Afr') & (df_full['model'] == 'PGS') & (df_full['split'] == 'test')]

In [9]:
tissue_trait_dict = {'INI30120' : 'BLOOD', 'INI20030780': 'LIVER', 'INI50030700': 'KIDNEY', 'INI1003063': 'LUNG', 'INI10030620': 'KIDNEY'}

In [11]:
def improve_fold(trait, tissue_trait_dict, score, df):
    return score/df[(df['Tissue'] == tissue_trait_dict[trait]) & (df['#response'] == trait)]['eval']

In [12]:
filtered['Improvement'] = filtered.apply(lambda x: improve_fold(x['#response'], tissue_trait_dict, x['eval'], filtered), axis=1)

/tmp/ipykernel_253748/455381963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['Improvement'] = filtered.apply(lambda x: improve_fold(x['#response'], tissue_trait_dict, x['eval'], filtered), axis=1)


In [15]:
prism = filtered[filtered['Improvement'] == 1]

In [20]:
merged = prism[['#response', 'eval', 'l_eval', 'u_eval', 'P', 'n']].merge(polyfun[['pheno_id', 'eval', 'l_eval', 'u_eval', 'P', 'n']], suffixes=("_prism", "_polypred"), left_on='#response', right_on='pheno_id')

In [21]:
merged.to_csv('final_comparison.tsv', sep='\t', index=False, header=True)